### **Run queries using DuckDB**

In [37]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd
import numpy as np
import duckdb as db

#### Load data files saved from API

In [29]:
# collect parquet files
output_file = "../census_outputs/census_tract_2020.parquet/"

parq_files = [ os.path.join(output_file, file) for file in os.listdir(output_file) if ".parquet" in file ]

parq_files.sort()

In [38]:
# create DuckDB database
con = db.connect("census_tract.db")

# create table
for file in parq_files:
    if "census_tract" not in con.sql("SHOW TABLES;").df().name.tolist():
        con.execute(f"CREATE TABLE census_tract AS SELECT * FROM '{file}'")
        print("Created table census_tract")
        print(f"Inserted {file.split('/')[-1]} into census_tract table")

    else:
        con.execute(f"INSERT INTO census_tract SELECT * FROM '{file}'")
        print(f"Inserted {file.split('/')[-1]} into census_tract table")

Inserted part.0.parquet into census_tract table
Inserted part.1.parquet into census_tract table
Inserted part.10.parquet into census_tract table
Inserted part.100.parquet into census_tract table
Inserted part.101.parquet into census_tract table
Inserted part.102.parquet into census_tract table
Inserted part.103.parquet into census_tract table
Inserted part.104.parquet into census_tract table
Inserted part.105.parquet into census_tract table
Inserted part.106.parquet into census_tract table
Inserted part.107.parquet into census_tract table
Inserted part.108.parquet into census_tract table
Inserted part.109.parquet into census_tract table
Inserted part.11.parquet into census_tract table
Inserted part.110.parquet into census_tract table
Inserted part.111.parquet into census_tract table
Inserted part.112.parquet into census_tract table
Inserted part.113.parquet into census_tract table
Inserted part.114.parquet into census_tract table
Inserted part.115.parquet into census_tract table
Insert

In [39]:
# show row count
query = """
SELECT count(*)
FROM census_tract;
        """

con.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    207996096 │
└──────────────┘



In [40]:
# create table of unique variables

query = """
DROP TABLE IF EXISTS variables;

CREATE TABLE variables AS 
SELECT variable_code, variable_name, measurement, demographic_target, demographic
FROM census_tract
GROUP BY 1, 2, 3, 4, 5;
        """

con.execute(query)

print("Created table variables")

Created table variables


In [41]:
# show row count
query = """
SELECT count(*)
FROM variables;
        """

con.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1232 │
└──────────────┘



In [48]:
# create table of unique states

query = """
DROP TABLE IF EXISTS states;

CREATE TABLE states AS 
SELECT state_code, state_name
FROM census_tract
GROUP BY 1, 2;
        """

con.execute(query)

print("Created table states")

Created table states


In [53]:
# create table of unique tracts

query = """
DROP TABLE IF EXISTS tracts;

CREATE TABLE tracts AS 
SELECT geo_id, tract_name, tract, county, state_code 
FROM census_tract
GROUP BY 1, 2, 3, 4, 5;
        """

con.execute(query)

print("Created table tracts")

Created table tracts


In [63]:
# pivot census_tract and create tables for demographic_targets